### Week 2 : Images as Signals & Noise removal

#### Date : 14th October, 2021

* This lab session is  is broken down into two sections, the first is just a visual demonstration therefore all you need to do is copy & paste or type out the code from task 1 and run each section.

* The second part requires you to code your own example.

#### Fourier Transform:

* We can use Fourier transforms to decompose an image into its sine and cosine components, resulting in an image in its frequency domain rather than its spatial domain. This has many implications and applications which will be discussed in class.

* Today however we wanted to demonstrate visually this decomposition process. Please note that we are showing you the signal decomposition by taking a 1D slice of the image.

#### Task 1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the image
img = cv2.imread('../images/sample_image.jpg', cv2.COLOR_BGR2RGB)

In [ ]:
# convert to double and gray scale
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
img_gray = img_gray.astype(np.double) / 255

In [ ]:
plt.imshow(img_gray)
plt.colorbar()

In [ ]:
# Next, we take a sample of this image
row = img_gray.shape[1] // 2
x = img_gray[row, :]
plt.plot(x)
plt.title('Grey-level profile at ' + str(row))

Here, we plot the amplitude of these frequencies which is found by taking the absolute value of our FFT signal

In [ ]:
X = np.fft.fft(x)
N = len(x)
fig = plt.figure(figsize=(12, 10))
ax1 = plt.subplot(2, 2, 1)
ax1.title.set_text('Amplitudes as a function of frequency')
plt.plot(np.abs(X))
ax2 = plt.subplot(2, 2, 2)
ax2.title.set_text('Phase as a function of frequency')
theta = np.arctan2(np.imag(X), np.real(X))
plt.plot(theta)
ax3 = plt.subplot(2, 2, 3)
ax3.title.set_text('Real part as a function of frequency')
plt.plot(np.real(X))
ax4 = plt.subplot(2, 2, 4)
ax4.title.set_text('Imaginary part as a function of frequency')
plt.plot(np.imag(X))

Next we are going to see the individual components that make up this signal, showing just a sub sample of these. 

In [ ]:
N = len(X)
reconstruction = 0.
p = 2 * np.pi * np.arange(N) / N # 0 to 2*pi
check_points = [i for i in range(0, 4)] + [i for i in range(4, N - 4, 16)] + [i for i in range(N - 4, N)]

for k in range(0, N):
    # decompose X[k] into magnitude and phase
    a = np.abs(X[k])
    theta = np.arctan2(np.imag(X[k]), np.real(X[k]))
    phi = k * p + theta
    component = a * np.cos(phi) / N
    reconstruction += component
    if k in check_points:
        fig, ax = plt.subplots(2, 1, figsize=(6,8))
        
        ax[0].plot(component)
        ax[0].set_title('Spatial frequency ' + str(k) + ' cycles across the image')
        
        ax[1].plot(reconstruction, alpha=0.5, label='reconstruction')
        ax[1].plot(x, alpha=0.5, label='GT')
        ax[1].legend()
        ax[1].set_title('reconstruction until ' + str(k))
        plt.show()

### Task 2: Noise removal

* For those of you who have experienced taking images in low lighting conditions you will inevitably have come across issues with noise, this is usually caused by having to use a very high ISO level. 

* The ISO setting is used to control the speed of the camera by amplifying the sensitivity of the cameras sensor.
 
* Today we will code two simple techniques that can be used to remove or reduce this noise. This does come at a price however by removing some fine details, as you will see.

#### Median Filter

* Your first task is to write a Median Filter. To start off with I would advise that you work on gray scale images. Once you manage this you can then figure out how to apply this to a colour image.

* The median filter works by iterating over an image using a sliding window (you can set the size, I would recommend something like 3x3 or 5x5). Each pixel will be replaced with the median pixel value of the values in the box. 

* Given this below matrix

$$\begin{bmatrix} 128 & 50 & 120 \\ 10 & \color{Red}{214} & 150 \\ 127 & 157 & 137 \end{bmatrix}$$

* After the median filter, the 214 would be replaced by the median value - 128

* Please implement your code using numpy only - compare the result with opencv later


In [ ]:
# Answer
fig, ax = plt.subplots(1, 2, figsize=(6,10))
ax[0].imshow(img)
ax[0].set_title('GT')
# ax[1].imshow(your_image)
# ax[1].set_title('Median Blurred')

#### Adaptive Filter

* Once you have completed the median filter on a colour image applying an Adaptive filter will be very easy. 

* This time all you need to do is compute the new pixel taking its variance into account and thus helping to preserve edges and details. This can be done using the following formula:

* Mean: 

$$\mu_{i, j} = \frac{1}{|\mathcal{K}_{i,j}|}\sum_{n_1, n_2 \in \mathcal{K}_{i,j}} \mathcal{K}_{i,j}(n_1, n_2)$$




* Variance:

$$\sigma^2_{i, j} = \frac{1}{|\mathcal{K}_{i, j}|}\sum_{n_1, n_2 \in \mathcal{K}_{i,j}} (\mathcal{K}_{i,j}(n_1, n_2) - \mu_{i, j})^2$$

* Output Pixel:

$$I(i, j) = \mu_{i, j} + \max(\frac{\sigma^2_{i, j} - v^2_{i, j}}{\sigma^2_{i, j}}, 0)(\mathcal{K}_{i,j}(n_1, n_2)-\mu_{i, j})$$   


where $\mathcal{K}$ is a (n, n) kernel. $v^2$ is the noise variance and can be eiter specified by the user or taken as the average of all local estimated variances

Question: Think about why we need $\max(\frac{\sigma^2_{i, j} - v^2_{i, j}}{\sigma^2_{i, j}}, 0)$ instead of using $\frac{\sigma^2_{i, j} - v^2_{i, j}}{\sigma^2_{i, j}}$？

In [ ]:
formula_img = cv2.imread('../images/formula_adaptive_filter.png')
plt.title('Adaptive Filter Formula')
plt.imshow(formula_img)
plt.axis('off')

* Please refer to the commented matlab code

In [ ]:
# function outimg = adaptiveFilter(img, varargin)
# [sx, sy, ~] = size(img);
# %create output image
# outimg = zeros(sx,sy,1);

# if length(varargin) > 1
#     GRID_SIZE = varargin{1};
# else
#     GRID_SIZE = 3;
# end

# paddedimg = padarray(img, [floor(GRID_SIZE/2) floor(GRID_SIZE/2)], 'replicate');

# if length(varargin) > 1
#     v  = varargin{2};
# else
#     %v  = 5;
#     temp = zeros(sx,sy);
#     for i = round(GRID_SIZE/2):(sx+floor(GRID_SIZE/2))
#         for j = round(GRID_SIZE/2):(sy+floor(GRID_SIZE/2))
#             list_of_pixels = paddedimg(i-floor(GRID_SIZE/2):i+floor(GRID_SIZE/2) ...
#             , j-floor((GRID_SIZE)/2):j+floor((GRID_SIZE)/2), :);
#              %find intensity values
#              list_of_pixels = list_of_pixels(:);
#              temp(i,j) = var(double(list_of_pixels));
#         end
#     end
#     v = mean(temp(:));
# end




# %iterate through all of the pixels
# for i = round(GRID_SIZE/2):(sx+floor(GRID_SIZE/2))
#     for j = round(GRID_SIZE/2):(sy+floor(GRID_SIZE/2))
#         %use our grid to find median result
#         list_of_pixels = paddedimg(i-floor(GRID_SIZE/2):i+floor(GRID_SIZE/2) ...
#             , j-floor((GRID_SIZE)/2):j+floor((GRID_SIZE)/2), :);
#         %find intensity values
#         list_of_pixels = list_of_pixels(:);
#         mean_pixel = mean(list_of_pixels);
#         var_pixel = var(double(list_of_pixels));
#         outimg(i-floor(GRID_SIZE/2),j-floor(GRID_SIZE/2)) = mean_pixel + ( (var_pixel - v)/var_pixel )* (paddedimg(i,j) - mean_pixel);
#     end
# end

# end


In [ ]:
# One answer
fig, axes = plt.subplots(1,2,figsize=(6,10))
axes[0].imshow(img)
axes[0].set_title('GT')
# axes[1].imshow(your adaptive image)
# axes[1].set_title('Adaptive Filter')

#### Filtering vectorized

* Using as little number of for-loops as possible, perform median filtering on the colour input.

* The vectorized down-sampling code from last week could be a good start. Try to pre-calculate the filter kernel (the array of weights) and use the .* operator to multiply 1-D arrays with each other.

* How much space does your method need for an 11x11 median filtering on a 3 channel (uint8)image? 

* Using the package - *time* does your algorithm compare to the built-in median filtering of opencv? Also, measure the difference between your implementation and opencv